In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pywaffle import Waffle
from pypalettes import load_cmap
from pyfonts import load_google_font
from drawarrow import ax_arrow, fig_arrow
from highlight_text import fig_text

In [ ]:
prizes = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/main/data/2025/2025-10-28/prizes.csv")

In [ ]:
prizes

In [ ]:
prizes["prize_decade"] = (prizes["prize_year"] // 10) * 10

prizes_agg = (
    prizes.groupby(["prize_decade", "ethnicity_macro"], as_index=False)
    .size()
    .pivot(index="prize_decade", columns="ethnicity_macro", values="size")
    .fillna(0)
    .reset_index()
    .melt(id_vars="prize_decade", var_name="ethnicity_macro", value_name="size")
)

prizes_agg

In [ ]:
n_colors = prizes_agg["ethnicity_macro"].nunique()
colors = load_cmap("CrystalGems", keep_first_n=n_colors, shuffle=3).colors + ["white"]

max_year_value = prizes_agg[prizes_agg["prize_decade"] == 2010]["size"].sum()

ncols = prizes_agg["prize_decade"].nunique()
fig, axs = plt.subplots(ncols=ncols, figsize=(25, 10))

for year, ax in zip(prizes_agg["prize_decade"].unique(), axs):
    values = list(prizes_agg[prizes_agg["prize_decade"] == year]["size"].values)
    values = sorted(values, reverse=True)
    values.append(max_year_value - sum(values))

    Waffle.make_waffle(
        ax=ax,
        rows=50,
        columns=10,
        values=values,
        vertical=True,
        colors=colors,
    )

    ax.text(x=0.1, y=-0.04, s=f"{int(year)}", fontsize=14, ha="center")

plt.show()